

# **Spotify Song Suggester using Cosine Similarity**








#Imports

In [ ]:
import pandas as pd
import numpy as np
from sklearn import preprocessing as pre
from sklearn.metrics.pairwise import cosine_similarity as cs

# Preprocessing

In [ ]:
#importing files
full_data = pd.read_csv("https://raw.githubusercontent.com/aymenboustani/MAIS-202-Deliverable-1/main/data/data.csv")
metadata = pd.DataFrame({'popularity': full_data['popularity'], 'id': full_data["id"], 
                         'release_date': full_data["release_date"], 'year': full_data["year"], 
                         'explicit': full_data["explicit"],'song_title': full_data["name"].str.lower()})
full_data = full_data.drop(columns = ['key', 'release_date','popularity','year','explicit'])

In [ ]:
#view some of the data
full_data

,acousticness,artists,danceability,duration_ms,energy,id,instrumentalness,liveness,loudness,mode,name,speechiness,tempo,valence
0,0.991000,['Mamie Smith'],0.598,168333,0.224,0cS0A1fUEUd1EW3FcF8AEI,0.000522,0.3790,-12.628,0,Keep A Song In Your Soul,0.0936,149.976,0.6340
1,0.643000,"[""Screamin' Jay Hawkins""]",0.852,150200,0.517,0hbkKFIJm7Z05H8Zl9w30f,0.026400,0.0809,-7.261,0,I Put A Spell On You,0.0534,86.889,0.9500
2,0.993000,['Mamie Smith'],0.647,163827,0.186,11m7laMUgmOKqI3oYzuhne,0.000018,0.5190,-12.098,1,Golfing Papa,0.1740,97.600,0.6890
3,0.000173,['Oscar Velazquez'],0.730,422087,0.798,19Lc5SfJJ5O1oaxY0fpwfh,0.801000,0.1280,-7.311,1,True House Music - Xavier Santos & Carlos Gomi...,0.0425,127.997,0.0422
4,0.295000,['Mixe'],0.704,165224,0.707,2hJjbsLCytGsnAHfdsLejp,0.000246,0.4020,-6.036,0,Xuniverxe,0.0768,122.076,0.2990
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174384,0.009170,"['DJ Combo', 'Sander-7', 'Tony T']",0.792,147615,0.866,46LhBf6TvYjZU2SMvGZAbn,0.000060,0.1780,-5.089,0,The One,0.0356,125.972,0.1860
174385,0.795000,['Alessia Cara'],0.429,144720,0.211,7tue2Wemjd0FZzRtDrQFZd,0.000000,0.1960,-11.665,1,A Little More,0.0360,94.710,0.2280
174386,0.806000,['Roger Fly'],0.671,218147,0.589,48Qj61hOdYmUCFJbpQ29Ob,0.920000,0.1130,-12.393,0,Together,0.0282,108.058,0.7140
174387,0.920000,['Taylor Swift'],0.462,244000,0.240,1gcyHQpBQ1lfXGdhZmWrHP,0.000000,0.1130,-12.077,1,champagne problems,0.0377,171.319,0.3200


In [ ]:
songset = full_data.copy()
songset = songset.drop(columns=['artists','name'])
songset = songset[['id','acousticness','danceability','duration_ms','energy','instrumentalness','liveness','loudness','mode','speechiness','tempo','valence']]
songset.iloc[:,1:]
x = pre.MinMaxScaler().fit_transform(songset.iloc[:,1:])
x

array([[0.99497992, 0.60526316, 0.03063657, ..., 0.09639547, 0.61590016,
        0.634     ],
       [0.64558233, 0.86234818, 0.02723665, ..., 0.05499485, 0.35682342,
        0.95      ],
       [0.99698795, 0.6548583 , 0.0297917 , ..., 0.1791967 , 0.40080983,
        0.689     ],
       ...,
       [0.80923695, 0.6791498 , 0.03997664, ..., 0.02904222, 0.44375726,
        0.714     ],
       [0.92369478, 0.46761134, 0.04482405, ..., 0.03882595, 0.70354856,
        0.32      ],
       [0.23995984, 0.68522267, 0.03614472, ..., 0.02657055, 0.46079989,
        0.747     ]])

### helper function

In [ ]:
#input an ID and it returns the index of corresponding song in the dataset
def searchbyID(ID):
  for i in range(len(full_data)):
    #print(full_data.iloc[20128,5])
    if full_data.iloc[i,5] == ID:
      return i
  return -1

## Input song IDs in the function to find similar songs

In [ ]:


# input ids of which you want to find similar songs to; and this outputs IDs of similar songs
def findSimilarSongs(songIDs):
  indices = []
  for ID in songIDs:
    indices.append(searchbyID(ID))

  sum = 0
  for index in indices:
    sum += x[index]

  meanvector = sum/len(indices)
  meanvector = np.array(meanvector).reshape(1,-1)


  similarity_vals = cs(x, meanvector)

  similar_songs_index = []
  for i in range(len(similarity_vals)):
    if similarity_vals[i] > 0.999:
      similar_songs_index.append(i)

  similar_songs_IDs = []
  for song_id in full_data.iloc[similar_songs_index]['id']:
    
    similar_songs_IDs.append(song_id)
  
  return similar_songs_IDs 


'\nmeanvector =  x[-2] #champagne problems taylor swift\nmeanvector = np.array(meanvector).reshape(1,-1)\n# meanvector = meanvector.reshape(1, -1)\n# def get_similar_vectors(remaining_dataset, avg_vector, how_many):\n#   similar_vector_uid = []\n#   uids = remaining_dataset[:,[0]]\n#   vecs = np.delete(remaining_dataset, 0, 1)\n#   similarity_vals = cs(vecs, avg_vector)\n#   similarity_with_uid = np.concatenate(uids, similarity_vals)\n#   sorted_array = similarity_with_uid[numpy.argsort(similarity_with_uid[:, 1])]\n#   return np.delete(sorted_array[(-1*how_many)-1:-1], 0, 1)\n\n# x.shape\nsimilarity_vals = cs(x, meanvector)\n\ncommon = []\nfor i in range(len(similarity_vals)):\n  if similarity_vals[i] > 0.999:\n    common.append(i)\nprint(common)\n'

### Testing the code: inputed 3 different song IDs and look at the similar ones

In [ ]:

findSimilarSongs(['0sY6ZUTh4yoctD8VIXz339','78DwRIo6Vj6wqteyerGetr','1gcyHQpBQ1lfXGdhZmWrHP'])

['1SecSXFtyYktKYc0qimTmw',
 '0sY6ZUTh4yoctD8VIXz339',
 '2EliNOReCRCe1ynR6G8yeT',
 '7u17w29WWTiftzGHs8rhpw',
 '78DwRIo6Vj6wqteyerGetr',
 '2fWVXXNvMNBbqhAiQCdZ2Z',
 '73e4ndrkfGIxqXAWm9PFvx',
 '6rap8NpGq8LDcVaJWkKFPn',
 '1gcyHQpBQ1lfXGdhZmWrHP']

In [ ]:
full_data.iloc[common]

,acousticness,artists,danceability,duration_ms,energy,id,instrumentalness,liveness,loudness,mode,name,speechiness,tempo,valence
5113,0.939,"['Ethel Merman', 'Robert Middleton', 'Marty Ma...",0.418,194400,0.225,1SecSXFtyYktKYc0qimTmw,0.000000,0.1380,-11.651,1,Moonshine Lullaby,0.0587,168.349,0.326
19922,0.920,['Taylor Swift'],0.462,244000,0.240,0sY6ZUTh4yoctD8VIXz339,0.000000,0.1130,-12.077,1,champagne problems,0.0377,171.319,0.320
20128,0.921,['Taylor Swift'],0.462,244000,0.242,2EliNOReCRCe1ynR6G8yeT,0.000000,0.1130,-12.077,1,champagne problems,0.0376,171.352,0.320
26867,0.905,['Ray Charles'],0.483,197173,0.187,7u17w29WWTiftzGHs8rhpw,0.000004,0.1220,-13.886,1,Funny (But I Still Love You),0.0439,179.031,0.338
36520,0.958,['John Mayer'],0.496,198200,0.238,78DwRIo6Vj6wqteyerGetr,0.010900,0.0875,-11.026,1,The Heart of Life,0.0301,175.960,0.349
63870,0.974,['The Kossoy Sisters'],0.449,176467,0.212,2fWVXXNvMNBbqhAiQCdZ2Z,0.000000,0.1150,-9.733,1,Little Birdie,0.0403,182.801,0.334
65632,0.925,['Odetta'],0.414,311773,0.215,73e4ndrkfGIxqXAWm9PFvx,0.000869,0.1090,-14.850,1,With God On Our Side,0.0500,167.912,0.335
158571,0.920,['Taylor Swift'],0.463,244000,0.248,6rap8NpGq8LDcVaJWkKFPn,0.000000,0.1130,-12.036,1,champagne problems,0.0374,171.346,0.325
174387,0.920,['Taylor Swift'],0.462,244000,0.240,1gcyHQpBQ1lfXGdhZmWrHP,0.000000,0.1130,-12.077,1,champagne problems,0.0377,171.319,0.320


# Now using Spotify API
#### Search for keywords in the spotify database for it to output the best matching tracks.
#### A predict function which uses FindSimilarSongs and directly outputs the corresponding tracks


In [ ]:
!pip install spotipy

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

cid = '1e112a2ea1a645fcba890f76298f0d2a'
secret = '9f264ec7484642a38f77ecb85d9bac36'
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

     |████████████████████████████████| 61kB 4.8MB/s 
     |████████████████████████████████| 153kB 12.8MB/s 
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.25.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [ ]:
search = 'call' #search in searchbar 
def searchInDatabase(search):
  output_raw = sp.search(search, limit=10)
  output = {'songs':[]}
  for i in range(len(output_raw['tracks']['items'])): 
    if searchbyID(output_raw['tracks']['items'][i]['id']) != -1:
      output["songs"].append({'title': output_raw['tracks']['items'][i]['name'],
                              'artist':output_raw['tracks']['items'][i]['artists'][0]['name'],
                              'id': output_raw['tracks']['items'][i]['id'], 
                              'image':output_raw['tracks']['items'][i]['album']['images'][0]['url']})
  return output


{'songs': [{'artist': 'The Weeknd',
   'id': '09mEdoA6zrmBPgTEN5qXmN',
   'image': 'https://i.scdn.co/image/ab67616d0000b2731f6a2a40bb692936879db730',
   'title': 'Call Out My Name'},
  {'artist': 'Bad Bunny',
   'id': '2TH65lNHgvLxCKXM3apjxI',
   'image': 'https://i.scdn.co/image/ab67616d0000b2734aef420e62863ebf622c27f5',
   'title': 'Callaita'},
  {'artist': 'Arctic Monkeys',
   'id': '086myS9r57YsLbJpU0TgK9',
   'image': 'https://i.scdn.co/image/ab67616d0000b2734ae1c4c5c45aabe565499163',
   'title': "Why'd You Only Call Me When You're High?"}]}

In [ ]:
# function that takes an array of song IDs and returns predictions for that ID 
def predict(songIDs):
  tracks = []
  similar_IDs = findSimilarSongs(songIDs)
  # print(similar_IDs)
  # print(sp.track(similar_IDs[0]))
  # for ID in similar_IDs:
  #  tracks.append(sp.track(ID))
  output_raw = sp.tracks(similar_IDs)
  output = {'songs':[]}

  for i in range(len(output_raw['tracks'])):
    output["songs"].append({'title': output_raw['tracks'][i]['name'],'artist': output_raw['tracks'][i]['artists'][0]['name'], 'id': output_raw['tracks'][i]['id'], 'image': output_raw['tracks'][i]['album']['images'][0]['url']})


  #for i in range(len(output['tracks']['items'])): 
  #  print(i)
  return output
  #return tracks

  #from outputed song, return the json format (just song, name etc from above)
predict(['0sY6ZUTh4yoctD8VIXz339'])


{'songs': [{'artist': 'Ethel Merman',
   'id': '1SecSXFtyYktKYc0qimTmw',
   'image': 'https://i.scdn.co/image/ab67616d0000b2734eb4133627a0474424266142',
   'title': 'Moonshine Lullaby'},
  {'artist': 'Taylor Swift',
   'id': '0sY6ZUTh4yoctD8VIXz339',
   'image': 'https://i.scdn.co/image/ab67616d0000b27333b8541201f1ef38941024be',
   'title': 'champagne problems'},
  {'artist': 'Taylor Swift',
   'id': '2EliNOReCRCe1ynR6G8yeT',
   'image': 'https://i.scdn.co/image/ab67616d0000b2731b5c73a284593a6e226f9d72',
   'title': 'champagne problems'},
  {'artist': 'Ray Charles',
   'id': '7u17w29WWTiftzGHs8rhpw',
   'image': 'https://i.scdn.co/image/ab67616d0000b273f0e951707ca49b533fdbf64e',
   'title': 'Funny (But I Still Love You)'},
  {'artist': 'John Mayer',
   'id': '78DwRIo6Vj6wqteyerGetr',
   'image': 'https://i.scdn.co/image/ab67616d0000b2737af5fdc5ef048a68db62b85f',
   'title': 'The Heart of Life'},
  {'artist': 'The Kossoy Sisters',
   'id': '2fWVXXNvMNBbqhAiQCdZ2Z',
   'image': 'https://